In [1]:
%matplotlib inline
import os
import csv
import numpy as np
import pandas as pd
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby

#import distance
import nltk
#nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from openpyxl import load_workbook

In [20]:
dict_pictureTrial = {
    '2019-02-14-14-28-49_1' : 2,
    '2019-02-14-14-45-49_2' : 2,
    '2019-02-06-12-37-45_2': 0,
    '2019-01-29-14-19-26_1': 2,
    '2019-01-29-14-40-36_2': 4,
    '2019-01-30-15-04-30_1': 2,
    '2019-03-12-09-30-5_1': 2,
    '2019-01-30-10-20-32_1': 7,
    '2019-01-29-13-25-4_1': 3,
    '2019-01-29-13-43-50_2': 2,
    '2019-03-07-16-17-30_1': 2,
    '2019-03-07-16-44-5_2': 3,
    '2019-03-14-13-56-56_2': 3,
    '2019-01-30-11-57-3_2': 2    
}


list_noPictureTrialExists = ['2019-02-18-10-28-35_2']

In [3]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    #'2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [4]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [5]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [7]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [8]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [9]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [10]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [11]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [12]:
def scratchPadPhraseEdit(phraseUser, subjName, full_path, picture):
    phraseUserEnd = list()
    
    
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if phraseUser[row_ind-1][1] != 'scratchPadText':
                if len(phraseUser[row_ind-1][1])>2: # this also removes any answers on the difficulty of the sentence
                    phraseUserEnd.append(phraseUser[row_ind-1])
                    #print(phraseUser[row_ind-1])
                    
    #print(' (1) phrases reduced to :       ', phraseUserEnd)
    # remove first two trials of baseline question and text composition
    if 'not_described' in picture or '2019-01-14-14-58-30' in full_path or '2019-02-06-12-37-45_2' in full_path or '2019-02-18-10-28-35_2' in full_path: 
        # yss has not copied one of the sentences, aq_session1_2 has not written the baseline question, ls2_session4_2 did not describe the picture
        del phraseUserEnd[0]
    else:
        del phraseUserEnd[0:2]
      
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_phraseUser:
        index_to_be_removed = dict_phraseUser[session_folder_name]
        #print('session in user phrases found')
    else:
        index_to_be_removed = []
        
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del phraseUserEnd[index]
        
    
    #print(' (2) phrases reduced to:       ', phraseUserEnd)
    
    # remove the initial rating of complexity, if they have written it:
    for index in range(0,len(phraseUserEnd)):
        sentence = phraseUserEnd[index][1]
        if sentence[0].isdigit():
            
            # if there is also a space after the digit:
            
            if sentence[1] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][1:]
            
        elif sentence[1].isdigit():
            if sentence[2] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][3:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
    
    #print(' (3) phrases reduced to:       ', phraseUserEnd)
    
    return phraseUserEnd

In [13]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [14]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
        
    return trials    

In [15]:
def getPictureTrial_writingTime(KeysSelected_new, userKeys):

    KeysSelected = [key[1] for key in KeysSelected_new]
    KeysSelectedTimeStr = [key[0] for key in KeysSelected_new]
    KeysSelectedTime = timeConversion(KeysSelectedTimeStr)
    
    userKeysTimeStr = [key[0] for key in userKeys]
    userKeysTime = timeConversion(userKeysTimeStr)
    
    NextPhraseInd = [i for i, x in enumerate(KeysSelected) if x == "NextPhrase"]
    
    time1, t1, t2 = KeysSelected_new[NextPhraseInd[0]][0].partition('+')
    BaselineEndTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time1, t1, t2 = KeysSelected_new[NextPhraseInd[1]][0].partition('+')
    PictureEndTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    PictureEndInd = KeysSelectedTime.index(PictureEndTime)
    
    BaselineEndInd = KeysSelectedTime.index(BaselineEndTime)
    
    pictureTrial = KeysSelected[BaselineEndInd:PictureEndInd]
    
    for i, key in enumerate(pictureTrial):
        if len(key) == 1:
            
            endReading_keyInd = BaselineEndInd + pictureTrial.index(key) - 1
            break
                   
    userKeyTime, userKeyInd = nearestTimePoint(userKeysTime, KeysSelectedTime[endReading_keyInd+1])
        
    # remove the dwell time from end of selecting the first key (letter/number)
    PictureStartTime = KeysSelectedTime[endReading_keyInd+1]-datetime.timedelta(milliseconds=\
                                    float(userKeys[userKeyInd][-1]))
        
    # If index matters in keysSelected matters, then the following is selected as the starting point of writing the
    # picture trial, which is when the first letter is already selected
    #PictureStartTime = KeysSelectedTime[endReading_keyInd]
    
    return (PictureStartTime, PictureEndTime)
            

In [16]:
def Picture_LettersNotSelected(UserKeys, timePicture, phraseUser, full_path):
    
    session_folder_name = full_path.split('\\')[-1]
    
    UserKeysTimeStr = [val[0] for val in UserKeys]
    UserKeysTime = timeConversion(UserKeysTimeStr)
    
    phraseUserEnd = list()
    
    
    
    # Find the time when the phrase starts
    timePhraseStartsWUserKeys, ind_start = nearestTimePoint(UserKeysTime, timePicture[0])
    ind_end_time, ind_end = nearestTimePoint(UserKeysTime, timePicture[1])
    
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if phraseUser[row_ind-1][1] != 'scratchPadText':
                if len(phraseUser[row_ind-1][1])>2: # this also removes any answers on the difficulty of the sentence
                    phraseUserEnd.append(phraseUser[row_ind-1])


    if session_folder_name in dict_pictureTrial:
        phrasePicture = phraseUserEnd[dict_pictureTrial[session_folder_name]][1]
    else:
        phrasePicture = phraseUserEnd[1][1]
    
    print('')
    print(phrasePicture)
    print('')
    
    flag_start = 0
    typing_list = list()
    letter_list = list()
        
        
    # collect the typing letters in a list of lists
    for ind_typing in range(ind_start, ind_end):
            
        if flag_start > 0:
            letter_list.append(UserKeys[ind_typing]) 
            if UserKeys[ind_typing][1] != UserKeys[ind_typing+1][1]:
                typing_list.append(letter_list)
                letter_list = list()
        else:
            letter_list.append(UserKeys[ind_typing])
            flag_start = 1
          
        
        
    # from the collected letters, find the ones that are not selected
    typing_list_temp = list()
        
    ind_letter = -1
    for letter in typing_list:
        ind_letter = ind_letter + 1
        typing_list_temp.append(letter[0][1])
        for instance in letter:
            if int(instance[2]) == 1:
                if len(typing_list_temp) > 2 and letter[0][1] == typing_list_temp[-3] and letter[0][2] > typing_list[ind_letter-2][-1][2]:
                    del typing_list_temp[-3]
                        
                del typing_list_temp[-1]
                    
                break
        
        
    ansr = len(typing_list_temp)/len(phrasePicture)
        
        
    
    return ansr

In [17]:
class DataForEverySession:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID], [self.blockNumber], [self.sessionNumber], [self.dataForTrial])), columns=['subjectID', 'block', 'session', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID], [self.blockNumber], [self.sessionNumber], [self.dataForTrial])), columns=['subjectID', 'block', 'session', self.variable])
        
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [24]:
metricComputed = 'AttendedButNotSelected'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'
resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Picture\Subject_Block_Session_Picture_' + \
metricComputed + '.xlsx'



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
        if '2019-02-27-15-08-32_1' in root: # picture not described in this folder, but in the next folder with 'picture'
            # in folder name
            continue
        
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None or keysSelected is None:
            continue
        else:
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            if sessionFolderName in list_noPictureTrialExists:  
                print('no picture trial exists')
                ansr = np.nan
            elif '2ndPart' in sessionFolderName:
                continue
            else:
                
                # fix scratchpad due to comma related file changes
                scratchPad_new = FixScratchPad(scratchPad)
            
                # fix userKeys due to comma related file changes
                userKeys_new = FixUserKeys(userKeys)
            
                # need to fix keys selected, where rows get combined because of an inverted comma
                keysSelected_new = FixKeysSelected(keysSelected)
            
                # find dwell time of typing
                userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
                # find the language of the data:
                if 'Hvad er det fulde navn på dit universitet?' in phraseLog[1][1]: 
                    language = 'Danish'
                else:
                    language = 'English'
                
                
                timePicture = getPictureTrial_writingTime(keysSelected_new, userKeys_wDwellTime)
            
            
                # Find letters not selected:
                # Find all keys that are not selected, just shortly looked at 
                ansr = Picture_LettersNotSelected(userKeys_wDwellTime, timePicture, scratchPad_new, root)
            
            
                
            dataToSave = DataForEverySession()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = ansr
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            dataToSave.AddToFile()
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1

People playing football on a grass field at night.

  subjectID block session  AttendedButNotSelected
0        ac     1       1                    0.66
subject path E:\Data\Data\ac\1\2019-02-11-11-33-32_2
subject and session name:  ac__1__2019-02-11-11-33-32_2

A guy doing extreme motorcycling in front of two other guys .

  subjectID block session  AttendedButNotSelected
0        ac     1       2                0.770492
subject path E:\Data\Data\ac\2\2019-02-12-15-00-04_1
subject and session name:  ac__2__2019-02-12-15-00-04_1

Two girls playing in a amusement park.

  subjectID block session  AttendedButNotSelected
0        ac     2       1                     0.5
subject path E:\Data\Data\ac\2\2019-02-12-15-27-37_2
subject and session name:  ac__2__2019-02-12-15-27-37_2

Motorcyclers in a field preparing for a race.

  subjectID block session  AttendedButNotSelected
0       


1 two guys skating wearing gloves and helmets 

  subjectID block session  AttendedButNotSelected
0        aq     2       2                0.717391
subject path E:\Data\Data\aq\3\2019-02-08-11-33-53_1stPart_1
subject and session name:  aq__3__2019-02-08-11-33-53_1stPart_1

2 a guy skiing and jumping over a barrel 

  subjectID block session  AttendedButNotSelected
0        aq     3       1                0.658537
subject path E:\Data\Data\aq\3\2019-02-08-12-11-34_2ndPart_1
subject and session name:  aq__3__2019-02-08-12-11-34_2ndPart_1
subject path E:\Data\Data\aq\3\2019-02-08-12-22-35_2
subject and session name:  aq__3__2019-02-08-12-22-35_2

1 Motorcyclers ready to start a race 

  subjectID block session  AttendedButNotSelected
0        aq     3       2                0.864865
subject path E:\Data\Data\aq\4\2019-02-13-11-05-20_1
subject and session name:  aq__4__2019-02-13-11-05-20_1

1a guy doing BMX 

  subjectID block session  AttendedButNotSelected
0        aq     4       1    


BMX kører laver et trick i det offentlige rum 

  subjectID block session  AttendedButNotSelected
0       bh2  5_MS       2                3.065217
subject path E:\Data\Data\cw\1\2019-02-07-13-20-50_1
subject and session name:  cw__1__2019-02-07-13-20-50_1

5The picture shows a participant of amountainbike tournament racing downhill .He is wearing red protective  gear and a white helmet. The race appears to take place in summer as the sun is shinning , the sky is blue and the grass is all dry. In the background,  other hills can be seen with forrests and a small village in the distance. The participant of the race is racing towards the finish line while a few spectators are watching.

  subjectID block session  AttendedButNotSelected
0        cw     1       1                1.110599
subject path E:\Data\Data\cw\1\2019-02-07-13-50-26_2
subject and session name:  cw__1__2019-02-07-13-50-26_2

3 The picture shows five players of rugby match. Three players are reaching for the ball,  two 


Several runners are sprinting for the end of a race.

  subjectID block session  AttendedButNotSelected
0        jm     5       1                0.192308
subject path E:\Data\Data\jm\5\2019-02-06-16-10-7_2
subject and session name:  jm__5__2019-02-06-16-10-7_2

Two football players are fighting for a ball.

  subjectID block session  AttendedButNotSelected
0        jm     5       2                0.133333
subject path E:\Data\Data\jp\1\2019-02-11-14-23-27_1
subject and session name:  jp__1__2019-02-11-14-23-27_1

5 To mænd spiller fodbold. Den ene bliver tacklet.

  subjectID block session  AttendedButNotSelected
0        jp     1       1                    1.52
subject path E:\Data\Data\jp\1\2019-02-11-14-46-36_2
subject and session name:  jp__1__2019-02-11-14-46-36_2

5 En blå bil kører på en smal grussti.

  subjectID block session  AttendedButNotSelected
0        jp     1       2                1.263158
subject path E:\Data\Data\jp\2_MS\2019-02-12-10-18-24_1
subject and session na


4 To fodboldspillere kæmper om en bold 

  subjectID block session  AttendedButNotSelected
0        le  5_MS       1                2.435897
subject path E:\Data\Data\le\5_MS\2019-03-04-12-00-15_2
subject and session name:  le__5_MS__2019-03-04-12-00-15_2

5 En fyr hænger i luften på sin tricykel,  mens han laver et trick.

  subjectID block session  AttendedButNotSelected
0        le  5_MS       2                2.328358
subject path E:\Data\Data\ls1\1\2019-02-11-10-18-57_1
subject and session name:  ls1__1__2019-02-11-10-18-57_1

Young man skateboarding in front of building and passing businessman.

  subjectID block session  AttendedButNotSelected
0       ls1     1       1                0.724638
subject path E:\Data\Data\ls1\1\2019-02-11-10-37-37_2
subject and session name:  ls1__1__2019-02-11-10-37-37_2

5 persons playing rugby in a park.

  subjectID block session  AttendedButNotSelected
0       ls1     1       2                0.411765
subject path E:\Data\Data\ls1\2\2019-02-12


4 4 kvinder på stranden der spiller volleyball.

  subjectID block session  AttendedButNotSelected
0        mh     5       2                0.468085
subject path E:\Data\Data\mn\1\2019-02-08-10-51-3_1stPart_1
subject and session name:  mn__1__2019-02-08-10-51-3_1stPart_1

En mand der hopper i en vandpyt

  subjectID block session  AttendedButNotSelected
0        mn     1       1                0.935484
subject path E:\Data\Data\mn\1\2019-02-08-11-05-7_2ndPart_1
subject and session name:  mn__1__2019-02-08-11-05-7_2ndPart_1
subject path E:\Data\Data\mn\1\2019-02-08-11-12-51_2
subject and session name:  mn__1__2019-02-08-11-12-51_2

6 To damer på rulleskøjter kæmper om hvem der kan komme først. De har begge en masse sikkerhedsudstyr på.

  subjectID block session  AttendedButNotSelected
0        mn     1       2                 0.47619
subject path E:\Data\Data\mn\2\2019-02-15-11-38-22_1
subject and session name:  mn__2__2019-02-15-11-38-22_1

4 En mand hopper på skateboard op ad en mur

subject path E:\Data\Data\rh\1\2019-03-07-16-44-5_2
subject and session name:  rh__1__2019-03-07-16-44-5_2

På billedet to slags dyr som løber hurtigt i skoven/marken. En løve (hunkøn) jagter en mørkebrun tyr.

  subjectID block session  AttendedButNotSelected
0        rh     1       2                0.950495
subject path E:\Data\Data\rh\2\2019-03-13-13-03-17_1
subject and session name:  rh__2__2019-03-13-13-03-17_1

Billedet viser en mand som er igang med at udføre et sjovt og spændende aktivitet på stranden.

  subjectID block session  AttendedButNotSelected
0        rh     2       1                0.478723
subject path E:\Data\Data\rh\2\2019-03-13-13-40-7_2
subject and session name:  rh__2__2019-03-13-13-40-7_2

På billedet er der to unge mænd som spiller hånbold. Der er en leder/træner bag dem som ser på dem. De ser alle glade ud.

  subjectID block session  AttendedButNotSelected
0        rh     2       2                0.743802
subject path E:\Data\Data\rh\3_MS\2019-03-14-13-30-3